In [11]:
import numpy as np
import pandas as pd
import hvplot.pandas
import yfinance as yf
import holoviews as hv

# Define the stock symbol and moving average window periods
ticker_symbol = 'AAPL'
short_window = 50
long_window = 100

# Download historical stock data using yfinance
stock = yf.Ticker(ticker_symbol)
stock_data = stock.history(period='5y')

# Calculate the short and long window simple moving averages
stock_data['SMA50'] = stock_data['Close'].rolling(window=short_window).mean()
stock_data['SMA100'] = stock_data['Close'].rolling(window=long_window).mean()

# Create a DataFrame to store the buy/sell signals
signals_df = pd.DataFrame(index=stock_data.index)
signals_df['Signal'] = 0.0  # Initialize with 0

# Generate the trading signal 0 or 1,
# where 1 is the short-window (SMA50) greater than the long-window (SMA100)
# and 0 is when the condition is not met
signals_df['Signal'][short_window:] = np.where(
    stock_data['SMA50'][short_window:] > stock_data['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time when the Signal value changes
# Identify trade entry (1) and exit (-1) points
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Create the plot for the stock's close prices
security_close = stock_data.hvplot.line(
    x='Date',
    y='Close',
    ylabel="Price in $",
    width=1000,
    height=400,
    title=f'{ticker_symbol} Stock Prices and SMA Crossovers'
)

# Create a line plot for SMA50
sma50_plot = stock_data.hvplot.line(
    x='Date',
    y='SMA50',
    ylabel="SMA50",
    width=1000,
    height=400,
    line_color='green',
    legend_label='SMA50'
)

# Create a line plot for SMA100
sma100_plot = stock_data.hvplot.line(
    x='Date',
    y='SMA100',
    ylabel="SMA100",
    width=1000,
    height=400,
    line_color='orange',
    legend_label='SMA100'
)

# Create a scatter plot for entry points
entry_plot = stock_data.loc[signals_df['Entry/Exit'] == 1.0].hvplot.scatter(
    x='Date',
    y='Close',
    color="purple",
    marker="^",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400,
    label="Entry"
)

# Create a scatter plot for exit points
exit_plot = stock_data.loc[signals_df['Entry/Exit'] == -1.0].hvplot.scatter(
    x='Date',
    y='Close',
    color="yellow",
    marker="v",
    size=200,
    legend=False,
    ylabel="Price in $",
    width=1000,
    height=400,
    label="Exit"
)

# Overlay the plots
overlay_plot = (security_close * sma50_plot * sma100_plot * entry_plot * exit_plot).opts(
    title=f'{ticker_symbol} - SMA Crossovers, Entry and Exit Points'
)

# Show the overlay plot
overlay_plot


:Overlay
   .Curve.I       :Curve   [Date]   (Close)
   .Curve.II      :Curve   [Date]   (SMA50)
   .Curve.III     :Curve   [Date]   (SMA100)
   .Scatter.Entry :Scatter   [Date]   (Close)
   .Scatter.Exit  :Scatter   [Date]   (Close)